# Object Pools

Let's start with a simple sketch, without worrying about type safety. All we really need, here, is a queue of objects. But when we pull objects from this pool, they need to come equipped with some means of rejoining the pool.

In [1]:
using DataStructures

In [2]:
mutable struct UnsafePool{T}
    q::Queue{T}
    UnsafePool{T}() where {T} = new(Queue{T}())
end

To do this, we'll wrap them in a struct that contains both the object and a reference back to the pool from whence it came.

In [3]:
mutable struct Recyclable{T}
    inner::Union{Nothing, T}
    home::Union{Nothing,Ref{UnsafePool{T}}}
end

In [4]:
function pull!(pool::UnsafePool{T})::Recyclable{T} where {T}
    item = dequeue!(pool.q)
    recyc = Recyclable{T}(item, Ref{UnsafePool{T}}(pool))
    return recyc
end

pull! (generic function with 1 method)

In [5]:
# for example
u = UnsafePool{Int}()
enqueue!(u.q, 1)
enqueue!(u.q, 2)
enqueue!(u.q, 3)
enqueue!(u.q, 4)
u

UnsafePool{Int64}(Queue{Int64}(Deque [[1, 2, 3, 4]]))

In [6]:
recyc = pull!(u)

Recyclable{Int64}(1, Base.RefValue{UnsafePool{Int64}}(UnsafePool{Int64}(Queue{Int64}(Deque [[2, 3, 4]]))))

In [20]:
function recycle!(r::Recyclable{T}) where {T}
    if r.home === nothing
        error("Already recycled $r")
    end
    enqueue!(r.home[].q, r.inner)
    r.home = nothing
    r.inner = nothing
    return
end

recycle! (generic function with 1 method)

In [10]:
# let's look at recyc, the wrapped object we pulled from the pool
println(recyc)

Recyclable{Int64}(1, Base.RefValue{UnsafePool{Int64}}(UnsafePool{Int64}(Queue{Int64}(Deque [[2, 3, 4]]))))


In [11]:
# and here's the pool it came from
println(u)

UnsafePool{Int64}(Queue{Int64}(Deque [[2, 3, 4]]))


In [12]:
# maybe we do some work on recyc
recyc.inner += 1
println(recyc)

Recyclable{Int64}(2, Base.RefValue{UnsafePool{Int64}}(UnsafePool{Int64}(Queue{Int64}(Deque [[2, 3, 4]]))))


In [13]:
# now we're done with it, so we recycle it
println("recyc = $recyc")
println("pool before recycling: $u")
recycle!(recyc)
println("pool after recycling: $u")



recyc = Recyclable{Int64}(2, Base.RefValue{UnsafePool{Int64}}(UnsafePool{Int64}(Queue{Int64}(Deque [[2, 3, 4]]))))
pool before recycling: UnsafePool{Int64}(Queue{Int64}(Deque [[2, 3, 4]]))
pool after recycling: UnsafePool{Int64}(Queue{Int64}(Deque [[2, 3, 4, 2]]))


In [14]:
println("recyc after recycling (throw this away): $recyc")

recyc after recycling (throw this away): Recyclable{Int64}(nothing, nothing)


## Let's try this with non-primitive objects now

In [23]:
array_pool = UnsafePool{Array{Float64,2}}()

UnsafePool{Array{Float64,2}}(Queue{Array{Float64,2}}(Deque [Array{Float64,2}[]]))

In [24]:
# now let's fill it up with a few expensively produced objects
for i in 1:4
    enqueue!(array_pool.q, rand(1024, 1024))
end
println(array_pool)

UnsafePool{Array{Float64,2}}(Queue{Array{Float64,2}}(Deque [[[0.3383317377996333 0.989097465464954 0.15537134445410206 0.14641760628514855 0.45430592291322225 0.2923071274397173 0.3010668326584387 0.33056474375358813 0.8585903105024184 0.12967576718882934 0.6766684890665768 0.6432909674685183 0.8602904449224846 0.8350393502339482 0.49907495152104286 0.6100080456302761 0.20376090532086666 0.057453758775921626 0.5194753331122037 0.717561174056708 0.6623220213382979 0.5949800436921895 0.6611946893072376 0.7207941562844309 0.3963131454954003 0.42020614379839505 0.09277811638853928 0.2094151079755482 0.03244082705180884 0.27203884328339356 0.42684355977750443 0.8527369423592921 0.7248045665129923 0.6512106582004218 0.6768335938638623 0.8593485976574586 0.8527023075709068 0.4064703498864517 0.23951149712162034 0.6311567599609915 0.40508972322336345 0.5442898872971107 0.2673671840381491 0.8669406814994896 0.36972860041380207 0.03271345823437688 0.26445075081961833 0.9040564280774033 0.6578078

057363881195006794 0.10333460010973194 0.7453719028602035 0.0032218312175438246 0.25095209068297253 0.6387409666434127 0.821486674801903 0.859114241620905 0.6226768755129164 0.40839979186278397 0.21197422507990105 0.6107217755604688 0.3140142417871823 0.9672854817975538 0.9686449168074791 0.5976846740954971 0.7782712249130228 0.3885294371282715 0.6768697867461633 0.31008043822575515 0.7851221732532807 0.2635233284560088 0.3531414639662429 0.08931608295055926 0.4636123184238099 0.22299644244645234 0.6650679467520453 0.011553454225236903 0.2709050959791386 0.37552598702163786 0.9875360557161823 0.14008600625598522 0.6391850252314579 0.04755490598979151 0.2678541381435757 0.015899077456858368 0.528911369793758 0.9972375929296309 0.4849437015125324 0.06177038198994045 0.9062060290350147 0.276562503979082 0.8564934790713341 0.08032622501527475 0.7863039362213917 0.8311414327046931 0.503279219396342 0.7663114947024201 0.25309381766187133 0.7798610052596024 0.14866246191880705 0.7729247383844

2824 0.4167304795054194 0.9454714365004471 0.836645124196342 0.22350628225965297 0.6542934080463763 0.37075946960050254 0.11237550007513786 0.38558511859095157 0.7309320333982414 0.4516669117668819 0.5346079383045368 0.41493178564934996 0.40095390637289086 0.894773465531244 0.9101250686309901 0.909937847517287 0.02873125191613446 0.3818607508027023 0.4701724368124902 0.05374887601929301 0.6517508287175982 0.7494553640821289 0.24030510447952103 0.6010378447631315 0.5941210787805291 0.11634420372449061 0.5554853500507673 0.6611712329960671 0.16388026951134482 0.5806876785685837 0.10896616675181314 0.4212043223534627 0.9184645200302988 0.09159503775880573 0.04818577337565877 0.3984330198692343 0.7976131877983712 0.4877170846233172 0.81680767184466; 0.5685941665872574 0.5222354161407818 0.8951406378212614 0.7244104100500746 0.6545902314851191 0.8664229854247156 0.23309665744881447 0.29862090920990547 0.45757296939073533 0.9059800727352059 0.38293622799705296 0.37703528269582276 0.359031256

 0.6911387313930211 0.9128848595003449 0.9696444159066389 0.014630289322460577 0.5754014014206945 0.4174925284230566 0.4110378390513951 0.41709901902561364 0.9622018955014204 0.16686192109551623 0.7689428451348752 0.9104883555035839 0.045060903615384174 0.7407464803272941 0.9231585568967691 0.09925597116912366 0.5057246394658281 0.7329331065232831 0.8141501242493889 0.962849841791495 0.4229898945301003 0.3976076411758538 0.6258256360901329 0.9600580583827081 0.8290697638486226 0.025987737244105524 0.3742808076893127 0.13826979192998934 0.39347206839663174 0.9621347835722565 0.11621681429346298 0.20617020262745944 0.2563676993823174 0.4010771623674698 0.9590470047680024 0.2963785632250102 0.18170965252221105 0.8559984332307047 0.05087470491335844 0.20572934961648404 0.4880856178308717 0.6581108585892812 0.32989294681008263 0.2540525689613691 0.8667186685864934 0.657808138283396 0.4556793451156771 0.3335907866975276 0.3244655471962006 0.7987449127168968 0.25531884770138547 0.060107520557

.864761707712677 0.3047043717616893 0.9091615975614076 0.11250975026790022 0.12200324792307793 0.8066194309685506 0.6231163095371948 0.021974667671736503 0.04559356060208586 0.7219272087394888 0.7882193310812504 0.584475342174398 0.6130852075490494 0.6955755856698447 0.6283567345133578 0.6052707116386948 0.1573687481487469 0.2751756919687556 0.35863940008723594 0.13488202030691188 0.9287506240719325 0.9371300563199345 0.29630638640687135 0.8463512079945934 0.25451478835331987 0.4258409070787432 0.2727542475129432 0.10529146435866243 0.5700306037931515 0.14260750142560163 0.4263873616767264 0.25872835247580483 0.3522929877663188 0.45639066263229444 0.3381436757432075 0.8453793841563804 0.30340724658946083 0.47709798182176777 0.11891447675609568 0.6169975521330009 0.2555494042525186 0.24438181339463627 0.5477007655185617 0.06562612379604316 0.6472790545786744 0.9014820966726544 0.7479159362364851 0.5711922685460635 0.9921079433975928 0.957224616770181 0.3691633115206576 0.098074278191258

9952416192563 0.9789699167491945 0.8757164431813298 0.36858056798271766 0.026403277765834732 0.6957170379023325 0.7173139203774352 0.724172260883605 0.44583807255533503 0.5654496453220283 0.5440883938945691 0.42193380662793545 0.09409799930098228 0.20431456748556265 0.5696374918843308 0.20989204438574793 0.7083797429846239 0.9881008333829144 0.7718792579930354 0.711922924661643 0.9802963650818599 0.8266594705506898 0.2723978702509098 0.5833381038626673 0.2561622455214061 0.9740238091958486 0.07966517683222185 0.5485274948368477 0.5974428009393415 0.6877197171502458 0.8245746210112546 0.7165241259230066 0.5111449335723492 0.81892574067714 0.6272454690775866 0.5715201959079452 0.3895881792169462 0.03974527087469282 0.3615103619594151 0.9918646591005127 0.16027432087470195 0.004443382432428811 0.1798489911540111 0.3612656645700103 0.06039902119378926 0.0032041615330402617 0.7112291358875236 0.3004046677200589 0.8905008630381168 0.17373894670845802 0.4146448065767794 0.11687836174227217 0.

Excessive output truncated after 524341 bytes.

In [25]:
# take a recyclable container from the pool\
ar = pull!(array_pool)
println("remaining in pool: $(length(array_pool.q))")
# do some work on it
println("max val in ar is $(maximum(ar.inner))")
# now return it to the pool
recycle!(ar)
println("remaining in pool: $(length(array_pool.q))")
println("ar = $ar")

remaining in pool: 3
max val in ar is 0.9999999167235263
remaining in pool: 4
ar = Recyclable{Array{Float64,2}}(nothing, nothing)


## Nicer syntax, approximating Python's `with` 

In [31]:
function from_pool(f::Function, pool::UnsafePool{T}) where {T}

    r = pull!(pool)
    try
        result = f(r.inner)
        return result
    catch e
        @warn(e.msg)
    finally
        println("recycling...")
        recycle!(r)
    end
end


from_pool (generic function with 1 method)

In [34]:
from_pool(array_pool) do array
    println("The maximum value in this array is $(maximum(array))")
end
    

The maximum value in this array is 0.9999996091471315
recycling...
